## 비지도학습 감성분석 - Lexicon 기반

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [9]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kimhansaem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/kimhansaem/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

### Wordnet Synset 및 Sentiwordnet Sentisynset 클래스

In [10]:
from nltk.corpus import wordnet

term = 'present'
synsets = wordnet.synsets(term)

In [11]:
type(synsets), len(synsets)

(list, 18)

In [12]:
print(synsets)

[Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [13]:
for synset in synsets[:5]: # 5개만 찍어보겟음
    print(f'#### name: {synset.name()} ####')
    print('POS:', synset.lexname())
    print('정의:', synset.definition())
    print('표제어:', synset.lemma_names())

#### name: present.n.01 ####
POS: noun.time
정의: the period of time that is happening now; any continuous stretch of time including the moment of speech
표제어: ['present', 'nowadays']
#### name: present.n.02 ####
POS: noun.possession
정의: something presented as a gift
표제어: ['present']
#### name: present.n.03 ####
POS: noun.communication
정의: a verb tense that expresses actions or states at the time of speaking
표제어: ['present', 'present_tense']
#### name: show.v.01 ####
POS: verb.perception
정의: give an exhibition of to an interested audience
표제어: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
#### name: present.v.02 ####
POS: verb.communication
정의: bring forward and present to the mind
표제어: ['present', 'represent', 'lay_out']


- 어휘간의 유사도

In [14]:
for synset in wordnet.synsets('tiger'):
    print(synset.name(), synset.definition())

tiger.n.01 a fierce or audacious person
tiger.n.02 large feline of forests in most of Asia having a tawny coat with black stripes; endangered


In [15]:
# 단어, 품사를 아는 경우에는 synset()
tiger = wordnet.synset('tiger.n.02')
tree = wordnet.synset('tree.n.01')
lion = wordnet.synset('lion.n.01')
cat = wordnet.synset('cat.n.01')
dog = wordnet.synset('dog.n.01')

In [16]:
# 단어간의 유사도
tiger.path_similarity(lion), tiger.path_similarity(dog)

(0.3333333333333333, 0.16666666666666666)

In [17]:
# 5개 단어간의 유사도
similarities = []
entities = [tree, lion, tiger, cat, dog]
for entity in entities:
    similarity = [entity.path_similarity(another) for another in entities]
    similarities.append(similarity)

In [18]:
df = pd.DataFrame(similarities, columns=['tree', 'lion', 'tiger', 'cat', 'dog'])
df

,tree,lion,tiger,cat,dog
0,1.000000,0.071429,0.071429,0.076923,0.125000
1,0.071429,1.000000,0.333333,0.250000,0.166667
2,0.071429,0.333333,1.000000,0.250000,0.166667
3,0.076923,0.250000,0.250000,1.000000,0.200000
4,0.125000,0.166667,0.166667,0.200000,1.000000


- SentiSynset 객체

In [24]:
import nltk
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/kimhansaem/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [25]:
from nltk.corpus import sentiwordnet

senti_synsets = sentiwordnet.senti_synsets('slow')


In [30]:
senti_synsets = sentiwordnet.senti_synsets('father')
print(type(senti_synsets))
print(senti_synsets)

<class 'filter'>


In [34]:
# father 단어의 긍정감성지수, 부정감정지수, 객관성 지수
father = sentiwordnet.senti_synset('father.a.01')
father.pos_score(), father.neg_score(), father.obj_score()

WordNetError: no lemma 'father' with part of speech 'a'

In [43]:
# mother 단어의 긍정감성지수, 부정감정지수, 객관성 지수
mother = sentiwordnet.senti_synset('mother.m.01')
mother.pos_score(), mother.neg_score(), mother.obj_score()

WordNetError: no lemma 'mother' with part of speech 'm'

In [35]:
# fabulous 단어의 긍정감성지수, 부정감정지수, 객관성 지수
fabulous = sentiwordnet.senti_synset('fabulous.a.01')
fabulous.pos_score(), fabulous.neg_score(), fabulous.obj_score()

(0.875, 0.125, 0.0)

In [36]:
# precisely 단어의 긍정감성지수, 부정감정지수, 객관성 지수
precisely = sentiwordnet.senti_synset('precisely.r.01')
precisely.pos_score(), precisely.neg_score(), precisely.obj_score()

(0.125, 0.0, 0.875)

In [45]:
wordnet.NOUN, wordnet.ADJ, wordnet.ADV, wordnet.VERB

('n', 'a', 'r', 'v')

- 감성지수 계산

In [50]:
from nltk import word_tokenize, pos_tag
sentence = "It;s good to see you again"
word_list = word_tokenize(sentence)

In [52]:
 import nltk
 nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kimhansaem/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [53]:
pos_tag(word_list)

[('It', 'PRP'),
 (';', ':'),
 ('s', 'RB'),
 ('good', 'JJ'),
 ('to', 'TO'),
 ('see', 'VB'),
 ('you', 'PRP'),
 ('again', 'RB')]

In [56]:
tag = ('good', 'JJ')
tag[1].startswith('J')

True

In [57]:
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('V'):
        return wordnet.VERB
    return None

In [58]:
for word, tag in pos_tag(word_list):
    print(word, tag)

It PRP
; :
s RB
good JJ
to TO
see VB
you PRP
again RB


In [62]:
# Sentence로부터 Senti_Synset 객체를 만드는 과정
sentence = "It's good to see you again."
word_list = [word for word in word_tokenize(sentence) if len(word) > 2]
word_list

['good', 'see', 'you', 'again']

In [63]:
for word, tag in pos_tag(word_list):
    wn_tag = penn_to_wn(tag)
    if wn_tag:
        synsets = list(sentiwordnet.senti_synsets(word, wn_tag))
        synset = synsets[0]
        print(synset)

<good.a.01: PosScore=0.75 NegScore=0.0>
<see.n.01: PosScore=0.0 NegScore=0.0>
<again.r.01: PosScore=0.0 NegScore=0.0>


In [64]:
sentiment = 0.
for word, tag in pos_tag(word_list):
    wn_tag = penn_to_wn(tag)
    if wn_tag:
        synsets = list(sentiwordnet.senti_synsets(word, wn_tag))
        synset = synsets[0]
        sentiment += synset.pos_score() - synset.neg_score()
sentiment

0.75

In [65]:
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [66]:
from nltk import sent_tokenize
document = "I watched this video at a friend's house. I'm glad I did not waste money buying this one. The video cover has a scene from the 1975 movie Capricorn One. The movie starts out with several clips of rocket blow-ups, most not related to manned flight. Sibrel's smoking gun is a short video clip of the astronauts preparing a video broadcast. He edits in his own voice-over instead of letting us listen to what the crew had to say. The video curiously ends with a showing of the Zapruder film. His claims about radiation, shielding, star photography, and others lead me to believe is he extremely ignorant or has some sort of ax to grind against NASA, the astronauts, or American in general. His science is bad, and so is this video."


In [67]:
sentiment = 0.0
for sentence in sent_tokenize(document):
    word_list = [word for word in word_tokenize(sentence) if len(word) > 2]
    for word, tag in pos_tag(word_list):
        wn_tag = penn_to_wn(tag)
        if wn_tag:
            lemma = lemmatizer.lemmatize(word, wn_tag)
            synsets = list(sentiwordnet.senti_synsets(lemma, wn_tag))
            if not synsets:
                print(word)
                continue
            synset = synsets[0]
            sentiment += synset.pos_score() - synset.neg_score()

print('긍정' if sentiment >=0 else '부정')

scene
blow-ups
Sibrel
voice-over
Zapruder
others
부정


- 감성을 계산해주는 함수

In [68]:
def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        word_list = [word for word in word_tokenize(raw_sentence) if len(word) > 2]
        tagged_sentence = pos_tag(word_list)
        for word, tag in tagged_sentence:
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wordnet.NOUN, wordnet.ADJ, wordnet.ADV, wordnet.VERB):
                continue                   
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wordnet.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = sentiwordnet.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    return 1 if sentiment >= 0 else 0

- IMDB 영화평 감성분석

In [78]:
df = pd.read_csv('data/labeledTrainData.tsv', sep='\t', quoting=3)      # 3: QUOTE_NONE
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [79]:
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />',' ')

In [80]:
# 구둣점, 숫자 제거 - 영문자가 아닌 글자는 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()

In [81]:
df.shape

(25000, 3)

In [82]:
#df = df.iloc[:1000, :]
#df.shape

In [ ]:
%time df['pred'] = df.review.apply(lambda x: swn_polarity(x))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(df.sentiment, df.pred)

## VADER Lexicon을 이용한 감성 분석

In [70]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kimhansaem/nltk_data...


True

In [83]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_score = senti_analyzer.polarity_scores(df.review[0])
senti_score

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}

In [84]:
def vader_polarity(document, threshold=0.1):
    score = senti_analyzer.polarity_scores(document)
    return 1 if score['compound'] >= threshold else 0

In [77]:
%time df['vader pred'] = df.review.apply(lambda x: vader_polarity(x, 0.1))

AttributeError: 'DataFrame' object has no attribute 'review'

In [86]:
accuracy_score(df.sentiment, df['vader pred'])

NameError: name 'accuracy_score' is not defined

- 예측 비교

In [85]:
cdf = df[['sentiment','pred','vader pred']]
cdf.head(10)

KeyError: "['pred', 'vader pred'] not in index"